# Pegando VLMO(CVM art°358) do repositório da CVM

Primeiramente devemos acessar o repositório da CVM:

http://dados.cvm.gov.br/dados/

Outra maneira de ser acessar estes dados é no rad.cvm.gov.br. Este sistema é capaz de buscar as informações e apresentá-las de maneira organizada. Porém ele não é muito simples para se fazer um webscrapping. Por isso optamos por importar os dados via CSV.
https://dados.cvm.gov.br/dataset/cia_aberta-doc-vlmo/resource/2beadbf7-1772-4537-865a-b0fb5dee4487
https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx
https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/VLMO/DADOS/vlmo_cia_aberta_2024.zip
        

## Download dos dados gerais em CSV

Download dos ZIPs que contêm as VLMOs no formato CSV.

In [1]:
import requests
import zipfile

def download():
    cvmzip_list = [
        'vlmo_cia_aberta_%d.zip' % y for y in range(2018, 2025)
    ]

    base_url = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/VLMO/DADOS/'

    for cvmzip in cvmzip_list:
        print('Arquivio:', cvmzip)
        response = requests.get(base_url + cvmzip)
        with open(cvmzip, 'wb') as fp:
            print('Download...')
            fp.write(response.content)

        with zipfile.ZipFile(cvmzip, 'r') as zip_ref:
            print('Unzip...')
            zip_ref.extractall()

download()

Arquivio: vlmo_cia_aberta_2018.zip
Download...
Unzip...
Arquivio: vlmo_cia_aberta_2019.zip
Download...
Unzip...
Arquivio: vlmo_cia_aberta_2020.zip
Download...
Unzip...
Arquivio: vlmo_cia_aberta_2021.zip
Download...
Unzip...
Arquivio: vlmo_cia_aberta_2022.zip
Download...
Unzip...
Arquivio: vlmo_cia_aberta_2023.zip
Download...
Unzip...


## Carregar os dados

Agora carregar os dados com o Pandas

In [1]:
import pandas as pd

# Função para carregar os dados de todos os anos em um dicionário de DataFrames
def carrega_dados():
    dados_por_ano = {}

    for ano in range(2018, 2025):
        nome_arquivo = f'vlmo_cia_aberta_con_{ano}.csv'
        try:
            df = pd.read_csv(nome_arquivo, delimiter=';', encoding='latin1')
            dados_por_ano[ano] = df
        except Exception as e:
            print(f"Erro ao ler o arquivo {nome_arquivo}: {e}")

    return dados_por_ano

# Função para extrair dados de uma empresa e ano específicos e salvar em um novo CSV
def extrair_dados_empresa(nome_empresa, ano_referencia):
    # Carregar os dados de todos os anos
    dados_por_ano = carrega_dados()

    # Verificar se o ano de referência está presente nos dados carregados
    if ano_referencia in dados_por_ano:
        # Obter o DataFrame correspondente ao ano de referência
        df_ano = dados_por_ano[ano_referencia]

        # Filtrar os dados da empresa específica para o ano especificado
        dados_empresa = df_ano[df_ano['Nome_Companhia'] == nome_empresa]

        # Verificar se há dados para a empresa especificada
        if not dados_empresa.empty:
            # Selecionar colunas desejadas para o novo DataFrame
            df_empresa = dados_empresa[['Tipo_Cargo', 'Tipo_Operacao', 'Volume']]

            # Salvar o DataFrame em um arquivo CSV com o nome da empresa e ano
            nome_arquivo = f"{nome_empresa}_{ano_referencia}_dados.csv"
            df_empresa.to_csv(nome_arquivo, index=False)
            print(f"Dados da empresa '{nome_empresa}' no ano '{ano_referencia}' salvos em '{nome_arquivo}'")
        else:
            print(f"Nenhum dado encontrado para a empresa '{nome_empresa}' no ano '{ano_referencia}'.")
    else:
        print(f"Ano '{ano_referencia}' não encontrado nos dados carregados.")

# Exemplo de uso:
nome_empresa = 'MARCOPOLO SA'  # Substitua pelo nome da empresa desejada
ano_referencia = 2024  # Substitua pelo ano desejado
extrair_dados_empresa(nome_empresa, ano_referencia)


Dados da empresa 'MARCOPOLO SA' no ano '2024' salvos em 'MARCOPOLO SA_2024_dados.csv'
